In [5]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
import keras

Using TensorFlow backend.


In [6]:
# Use header to load my csv file with variable names
header = ['label','rndm','weight']
# boosted frame
header += ['W_B_pt','W_B_eta','W_B_phi','W_B_E','W_B_m']
header += ['b_B_pt','b_B_eta','b_B_phi','b_B_E','b_B_m']
header += ['Wjj_B_dR','tWb_B_dR'] #dR = sqrt(phi^2 + eta^2)
header += ['Wjj_B_deta','Wjj_B_dphi','tWb_B_deta','tWb_B_dphi']
# lab frame
header += ['W_pt','W_eta','W_phi','W_E','W_m']
header += ['b_pt','b_eta','b_phi','b_E','b_m']
header += ['Wjj_dR','tWb_dR']
header += ['Wjj_deta','Wjj_dphi','tWb_deta','tWb_dphi']
header += ['t_pt','t_eta','t_phi','t_E','t_m']

header += ['btag1', 'btag2', 'btag3'] #binary representation of likelihood of the jet being an actual bjet

df = pd.read_csv('~/projects/top-reco-tests/samples/result.csv', names=header, delimiter=' ', skiprows=1)

In [7]:
df.head()

,label,rndm,weight,W_B_pt,W_B_eta,W_B_phi,W_B_E,W_B_m,b_B_pt,b_B_eta,...,tWb_deta,tWb_dphi,t_pt,t_eta,t_phi,t_E,t_m,btag1,btag2,btag3
0,1,-0.54,0.000011,48.501668,0.056284,2.524057,116.303751,105.672564,48.501668,-0.056284,...,-0.239658,-2.566564,100.524522,1.958788,2.919984,399.158659,164.944705,5.0,1.0,1.0
1,1,-0.54,0.000011,48.501668,0.056284,2.524057,116.303751,105.672564,48.501668,-0.056284,...,-0.239658,-2.566564,100.524522,1.958788,2.919984,399.158659,164.944705,5.0,1.0,1.0
2,0,-0.88,0.000010,123.407388,-0.826980,0.902965,188.335550,84.993206,123.407388,0.826980,...,-1.487057,2.487893,83.448000,1.877967,-0.577286,453.880273,357.798334,1.0,1.0,2.0
3,0,-0.88,0.000010,108.719836,-1.055445,0.910231,197.607526,91.572443,108.719836,1.055445,...,-1.871172,2.441210,83.123734,1.720035,-0.884576,444.189780,374.057018,1.0,5.0,2.0
4,0,-0.88,0.000010,125.274355,-0.900164,1.302688,186.303408,49.706508,125.274355,0.900164,...,-1.758664,-3.110917,6.357822,4.293132,-2.493616,434.704504,367.160877,1.0,5.0,1.0


In [8]:
#down-sample the class of non-jet samples to 1/4 of the original size (prevents model bias towards to majority class)
pos_class = df[df['label'] == 1]
neg_class = df[df['label'] == 0]
print(neg_class.shape)
neg_class = neg_class.sample(frac=0.3)
neg_class.shape
final_df = pd.concat([neg_class, pos_class])

(13759043, 43)


In [9]:
X_norm = pd.read_csv('~/projects/top-reco-tests/samples/norm_result.csv', delimiter=',')

In [10]:
X_norm.head()

,Unnamed: 0,weight,W_B_pt,W_B_eta,W_B_phi,W_B_E,W_B_m,b_B_pt,b_B_eta,b_B_phi,...,tWb_deta,tWb_dphi,t_pt,t_eta,t_phi,t_E,t_m,btag1,btag2,btag3
0,8795017,0.779540,0.098923,0.653887,0.783632,0.138814,0.180137,0.098923,0.698988,0.339211,...,0.665743,0.381083,0.745544,0.701901,0.095539,0.224708,0.101908,0.0,0.0,0.4
1,4403760,0.711833,0.805187,0.802556,0.595213,0.817569,0.816806,0.805187,0.530874,0.150813,...,0.815172,0.093566,0.344259,0.757124,0.792748,0.806848,0.825026,0.8,0.0,0.6
2,6540406,0.788310,0.124111,0.591804,0.991573,0.304417,0.380235,0.124111,0.767501,0.440593,...,0.602483,0.458784,0.276899,0.627529,0.466463,0.113498,0.232345,0.0,0.0,0.0
3,8743837,0.749267,0.252096,0.527352,0.899640,0.338698,0.298530,0.252096,0.812448,0.399795,...,0.468976,0.086738,0.108350,0.500023,0.075165,0.220002,0.387675,0.0,0.0,0.0
4,8352104,0.668700,0.313891,0.827911,0.786303,0.548906,0.359841,0.313891,0.462701,0.341881,...,0.836990,0.220159,0.249216,0.823278,0.185248,0.802398,0.681148,0.0,0.0,0.0


In [9]:
print(X_norm.shape)
print(y.shape)

(4513598, 42)
(5201550,)


In [13]:
pd.concat([X_norm, y], axis=1).shape

(8121223, 43)

In [10]:
#dataframe preprocessing
y = final_df['label']
X = final_df.drop('label', axis=1).drop('rndm', axis=1)

In [11]:
#data normalization
def normalize(col):
    print(col.name)
    threshold = col.quantile(0.9)
    mini = col.min()
    slopeUpper = (1 - 0.8) / (col.max() - threshold)
    slopeLower = (0.7 - 0) / (threshold - mini)
    def norm_helper(row):
        if row > threshold:
            return 0.8 + slopeUpper * (row - threshold)
        else:
            return 0 + slopeLower * (row - mini)
    return col.apply(norm_helper)

In [14]:
#normalize the data 
X_norm = X.apply(normalize, axis=0)

weight
weight
W_B_pt
W_B_eta
W_B_phi
W_B_E
W_B_m
b_B_pt
b_B_eta
b_B_phi
b_B_E
b_B_m
Wjj_B_dR
tWb_B_dR
Wjj_B_deta
Wjj_B_dphi
tWb_B_deta
tWb_B_dphi


/usr/common/software/tensorflow/intel-tensorflow/1.11.0-py36/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  


W_pt
W_eta
W_phi
W_E
W_m
b_pt
b_eta
b_phi
b_E
b_m
Wjj_dR
tWb_dR
Wjj_deta
Wjj_dphi
tWb_deta
tWb_dphi
t_pt
t_eta
t_phi
t_E
t_m
btag1
btag2
btag3


In [18]:
print(X_norm.shape)
print(pd.concat([X_norm, y], axis=1).shape)
pd.concat([X_norm, y], axis=1)

(5201550, 41)
(5201550, 42)


,weight,W_B_pt,W_B_eta,W_B_phi,W_B_E,W_B_m,b_B_pt,b_B_eta,b_B_phi,b_B_E,...,tWb_dphi,t_pt,t_eta,t_phi,t_E,t_m,btag1,btag2,btag3,label
12438338,0.800411,0.297075,0.599235,0.667350,0.165631,0.191580,0.297075,0.606868,0.278562,0.136466,...,0.053514,0.177841,0.385313,0.851404,0.460831,0.155635,0.000,0.000,0.700,0
985375,0.619238,0.295563,0.800228,0.475304,0.801708,0.804447,0.295563,0.510085,0.086502,0.361424,...,0.554557,0.231880,0.809815,0.193400,0.560870,0.638665,0.000,0.000,0.000,0
1569436,0.623071,0.172458,0.625551,0.248288,0.355552,0.442852,0.172458,0.581744,0.637144,0.079986,...,0.559691,0.250025,0.588869,0.041749,0.135312,0.262386,0.700,0.000,0.700,0
9130912,0.587012,0.227328,0.823879,0.533976,0.403411,0.165264,0.227328,0.449698,0.145178,0.800977,...,0.510086,0.510757,0.695884,0.342585,0.800414,0.528680,0.000,0.000,0.700,0
4049780,0.629594,0.275287,0.490318,0.569607,0.838021,0.840543,0.275287,0.804531,0.180812,0.395329,...,0.493672,0.356574,0.814389,0.273808,0.833373,0.834727,0.000,0.000,0.000,0
871714,0.611178,0.593305,0.623277,0.537321,0.292380,0.306324,0.593305,0.583915,0.148524,0.314619,...,0.197677,0.404003,0.813694,0.078990,0.812061,0.304010,0.000,0.000,0.000,0
6933878,0.586599,0.229429,0.660347,0.332576,0.095286,0.075721,0.229429,0.548525,0.855199,0.158729,...,0.205273,0.228567,0.513647,0.468049,0.065401,0.116576,0.000,0.000,0.000,0
4615089,0.620626,0.163176,0.669944,0.328915,0.173561,0.206487,0.163176,0.539362,0.845774,0.121184,...,0.196632,0.177607,0.441259,0.450647,0.166661,0.155474,0.000,0.700,0.000,0
6967156,0.664954,0.131573,0.564757,0.223202,0.100952,0.125942,0.131573,0.639784,0.612056,0.061910,...,0.223890,0.227860,0.558494,0.380411,0.036968,0.085764,0.000,0.000,0.350,0
5519162,0.643559,0.140219,0.647091,0.381885,0.187300,0.233627,0.140219,0.561180,0.982173,0.080310,...,0.384666,0.528764,0.479950,0.995088,0.262953,0.150050,0.000,0.000,0.525,0


In [ ]:
#convert normalized dataframe to CSV file so that we don't have to renormalize everytime
#X_norm.to_csv('~/projects/top-reco-tests/samples/norm_result.csv')

In [ ]:
#X_norm = pd.read_csv('~/projects/top-reco-tests/samples/norm_result.csv', delimiter=',')
#X_norm = X_norm.drop('Unnamed: 0', axis=1)
#X_norm.head()

In [9]:
#Randomly split the data for training and testing using sklearn's train_test_split method
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=.15, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=.15, shuffle=True)

In [10]:
#print out the number of events for the trianing, testing, and validation sets
print("X_train events: " + str(X_train.shape[0]))
print("X_test events: " + str(X_test.shape[0]))
print("X_val events: " + str(X_val.shape[0]))
print("Pos training events: " + str(sum(y_train == 1)))
print("Negative training events: " + str(sum(y_train == 0)))

X_train events: 3758119
X_test events: 780233
X_val events: 663198
Pos training events: 775853
Negative training events: 2982266


In [11]:
#calculate the % of samples that are positively labeled in the training data
print("Percent of positive class in the training set: {0}".format(sum(y_train == 1) / (sum(y_train == 1) + sum(y_train == 0))))

Percent of positive class in the training set: 0.20644716146561617


In [12]:
#Hyper-parameters
max_num_epochs = 15
batch_size = 512
learning_rate = 0.005
num_batches = int(len(X_train) / batch_size)

In [13]:
#Network Paramters
n_features = X_train.shape[1]
n_nodes_1 = 256
n_nodes_2 = 128
n_nodes_3 = 64
n_nodes_4 = 32
n_out = 2

In [14]:
#Create placeholders for the input data
X_tr = tf.placeholder(tf.float32, shape=[None, n_features])
y_tr = tf.placeholder(tf.float32, shape=[None, 2])
training = tf.placeholder(tf.bool, name='training')

In [15]:
#Build the graph
W1 =tf.Variable(tf.truncated_normal([n_features, n_nodes_1], stddev=0.1))
b1 = tf.Variable(tf.constant(0.1, shape=[1, n_nodes_1]))
batch_norm1 = tf.layers.batch_normalization(inputs=tf.matmul(X_tr, W1) + b1, training=training)
layer1 = tf.nn.relu(batch_norm1)

W2 = tf.Variable(tf.truncated_normal([n_nodes_1, n_nodes_2], stddev=0.1))
b2 = tf.Variable(tf.constant(0.1, shape=[1, n_nodes_2]))
batch_norm2 = tf.layers.batch_normalization(inputs=tf.matmul(layer1, W2) + b2, training=training)
layer2 = tf.nn.relu(batch_norm2)

W3 = tf.Variable(tf.truncated_normal([n_nodes_2, n_nodes_3], stddev=0.1))
b3 = tf.Variable(tf.constant(0.1, shape=[1, n_nodes_3]))
batch_norm3 = tf.layers.batch_normalization(inputs=tf.matmul(layer2, W3) + b3, training=training)
layer3 = tf.nn.relu(batch_norm3)

W4 = tf.Variable(tf.truncated_normal([n_nodes_3, n_nodes_4], stddev=0.1))
b4 = tf.Variable(tf.constant(0.1, shape=[1, n_nodes_4]))
layer4 = tf.nn.relu(tf.matmul(layer3, W4) + b4)

W_out = tf.Variable(tf.truncated_normal([n_nodes_4, n_out], stddev=0.1))
b_out = tf.Variable(tf.constant(0.1, shape=[1, n_out]))
logits = tf.matmul(layer4, W_out) + b_out

In [16]:
#Calculate the loss using Cross Entropy and define the optimizer
entropy = tf.nn.weighted_cross_entropy_with_logits(logits=logits, targets=y_tr, pos_weight=1.5)
loss = tf.reduce_mean(entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [17]:
#compute the accuracy using Area Under Curve -- this gives us a better metric for imbalanced datasets
prediction = tf.cast(tf.argmax(tf.nn.softmax(logits), axis=1), tf.float32)
actual = tf.cast(tf.argmax(y_tr, axis=1), tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, actual), tf.float32))
auc = tf.metrics.auc(labels=actual, predictions=prediction)

In [18]:
#create the session
sess = tf.InteractiveSession()
sess.run(tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()))
saver = tf.train.Saver()

In [19]:
#run the model on the input data and calculate/print validation and training accuracy for each epoch
epoch = 0
best_val_acc = 0
loss_decreased = False
loss_decrease_threshold = 0.1

while epoch < max_num_epochs:
    total_loss = 0
    for batch in range(num_batches):
        index = batch * batch_size
        last = index + batch_size
        #training
        X_batch, y_batch = X_train.iloc[index:last].values, tf.keras.utils.to_categorical(y_train.iloc[index:last])
        opt, batch_loss, acc, auc_score = sess.run([optimizer, loss, accuracy, auc], feed_dict={X_tr: X_batch, y_tr: y_batch, training: True})
        total_loss += batch_loss
    print("Epoch {0} ==> Acc:{1}, AUC: {2}, Loss: {3}".format(epoch, acc, float(auc_score[0]), total_loss))
    #test validation accuracy every 10 epochs
    if epoch % 5 == 0 and epoch != 0:
        val_acc, val_auc_score = sess.run([accuracy, auc], feed_dict={X_tr: X_val.values, y_tr: tf.keras.utils.to_categorical(y_val), training: False})
        print("Val Accuracy {0}, AUC score {1}".format(val_acc, val_auc_score[0]))
        
        #implement early stopping
        if (best_val_acc > (float(val_auc_score[0]) * (1 + loss_decrease_threshold))):
            if (loss_decreased):
                break
            else:
                loss_decreased = True
        else:
            best_val_acc = max(float(val_auc_score[0]), best_val_acc)
            loss_decreased = False
        
    epoch += 1;

Epoch 0 ==> Acc:0.890625, AUC: 0.7419406175613403, Loss: 9480.50166875124
Epoch 1 ==> Acc:0.90625, AUC: 0.7450562715530396, Loss: 9269.652017608285
Epoch 2 ==> Acc:0.9140625, AUC: 0.7464731335639954, Loss: 9208.080362603068
Epoch 3 ==> Acc:0.9140625, AUC: 0.7475801706314087, Loss: 9172.538533970714
Epoch 4 ==> Acc:0.90625, AUC: 0.7483936548233032, Loss: 9155.834634691477
Epoch 5 ==> Acc:0.9296875, AUC: 0.7489691376686096, Loss: 9144.962715655565
Val Accuracy 0.7934131026268005, AUC score 0.7489696145057678
Epoch 6 ==> Acc:0.8984375, AUC: 0.7432739734649658, Loss: 9131.600544527173
Epoch 7 ==> Acc:0.8828125, AUC: 0.7444036602973938, Loss: 9124.860549017787
Epoch 8 ==> Acc:0.8984375, AUC: 0.7453882694244385, Loss: 9113.73338843882
Epoch 9 ==> Acc:0.9296875, AUC: 0.7460806965827942, Loss: 9112.54985718429
Epoch 10 ==> Acc:0.921875, AUC: 0.7467236518859863, Loss: 9104.045041263103
Val Accuracy 0.7934131026268005, AUC score 0.7467238306999207


KeyboardInterrupt: 

In [20]:
#test the model on test data that was take from result.csv 
test_acc, predictions = sess.run([accuracy, prediction], feed_dict={X_tr: X_test.values, y_tr: tf.keras.utils.to_categorical(y_test), training: False})
print("Test Accuracy: {0}".format(test_acc))

Test Accuracy: 0.7936821579933167


In [29]:
#Print out the percent of values that were classified as tripets in the test set 
print("Percent classified as a triplet: {0}".format(sum(predictions) / X_test.shape[0]))
sum(predictions)

Percent classified as a triplet: 0.0


0.0

In [ ]:
#save the model
save_path = saver.save(sess, "models/model2.ckpt")
print("Model saved in path: %s" % save_path)

In [27]:
#test the model on specific instances.
index = 20067
pos_class_test = X_train.iloc[int(np.argwhere(y_train == 1)[index])]
print(sess.run([prediction], feed_dict={X_tr: np.array(pos_class_test).reshape(-1, 41), training: False}))
print(np.array(y_train.iloc[np.argwhere(y_train == 1)[index]])[0])

[array([0.], dtype=float32)]
1


In [ ]:
#saver.restore(sess, "models/model.ckpt")
#print("Model Restored")

In [ ]:
#close the current session and drop all saved variable values. --MAKE SURE TO SAVE FIRST--
sess.close()